Проект анализа для компании "Простые вещи"

В данном проекте я сделал следующее:

1. Выгрузил библиотеки  
2. Проверил данные 
3. Предообработал данные, удалив часть строчек 
4. Провел анализ данных
5. Провел когортный анализ 
6. Провел RFM анализ
7. Вычислил DAU, WAU, sticky 

1. Загрузка библиотек. 

In [1]:
import ipywidgets as widgets
from IPython.display import display
import pandas as pd
import streamlit as st
import numpy as np
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt

2. Загрузка и проверка данных

In [2]:
data = pd.read_excel('correct_payments.xlsx')


In [3]:
data.isna().sum()

Unnamed: 0               0
id                       0
action_date            652
bank                     0
site                     0
order_id              2867
customer                 1
type                     0
operation_sum            0
operation_currency       0
comission_sum           62
final_sum               62
final_currency          62
status                   0
aim                    673
comission_perc           0
file                     0
dtype: int64

Как видно, в данных много пропусков. Нужно посмотреть, с чем это связано 

In [4]:
data[data['action_date'].isna()]['status'].value_counts()

status
Отклонена    652
Name: count, dtype: int64

In [5]:
data[data['order_id'].isna()]['status'].value_counts()

status
Завершена    2160
Отклонена     707
Name: count, dtype: int64

In [6]:
data[data['final_sum'].isna()]['status'].value_counts()

status
Отклонена    62
Name: count, dtype: int64

In [7]:
data[data['aim'].isna()]['status'].value_counts()

status
Завершена    504
Отклонена    169
Name: count, dtype: int64

Как видно большая часть пропусков свзана с отклонением платежей

In [8]:
data['action_date'].value_counts()

action_date
                       169
2024-06-22 12:37:00      3
2024-06-22 10:26:00      3
2024-06-04 15:21:00      2
2024-06-22 11:27:00      2
                      ... 
2024-04-14 21:44:00      1
2024-04-14 21:21:00      1
2024-04-14 12:09:00      1
2024-04-14 11:16:00      1
2024-07-01 08:32:00      1
Name: count, Length: 3274, dtype: int64

In [9]:
data['status'].value_counts()

status
Завершена    3327
Отклонена     821
Name: count, dtype: int64

В датасете пропуски связаны с отклоненными транзакциями - их мы далее удалил

In [10]:
data['type'].value_counts()

type
Регулярная оплата              2858
Оплата                          836
Оплата с созданием подписки     454
Name: count, dtype: int64

Видно, что с регулярной оплатой пользователей гораздо больше

In [11]:
data.query('status == "Завершена" or status == "Completed"')['customer'].nunique()

1476

In [12]:
data['customer'].str.contains('nan').value_counts()

customer
False    4141
True        6
Name: count, dtype: int64

2. Предообратка данных

In [13]:
data[data['customer'].isna()]

,Unnamed: 0,id,action_date,bank,site,order_id,customer,type,operation_sum,operation_currency,comission_sum,final_sum,final_currency,status,aim,comission_perc,file
2562,332,2096037273,2024-03-05 13:58:00,Тинькофф,prostieveschi.ru,NaN,NaN,Регулярная оплата,100,RUB,3.9,96.1,RUB,Завершена,"Поддержите ""Простые вещи""",3.2,март.xls


Среди пользователей мы видим одного неизвестного. Чтобы он не потерялся, можно его переименовать 

In [14]:
# Найдём маску, где customer == NaN
mask = data['customer'].isna()

# Допустим, хотим всем таким строкам (если одна или несколько) присвоить одно имя:
data.loc[mask, 'customer'] = "Unknown_1"

# Теперь во всех строках, где раньше было NaN, в 'customer' появится "Unknown_1"


Как видим, в данных есть разные валюты. Их нужно свести к рублям 

In [15]:
data['operation_currency'].value_counts()

operation_currency
RUB    4122
EUR      15
USD       7
BYN       4
Name: count, dtype: int64

Переводим все данные в числа с десятиными дробями 

In [16]:
data['comission_sum'] = data['comission_sum'].astype(float)

In [17]:
data['final_sum'] = data['final_sum'].astype(float)

In [18]:
data['comission_perc'] = data['comission_perc'].astype(float)

Приводим все статусы к единому формату 

In [19]:
data['status'] = data['status'].replace({
    'Завершена': 'Completed',
    'Отклонена': 'Declined'
})

In [20]:
df_completed = data.query("status == 'Completed'")
df_completed['customer'].nunique()

1477

Удаляем лишние данные, т.е. отклоненные платежи 

In [21]:
data = data[data['status'] == 'Completed']
data = data[data['action_date'].notna()]
data = data[data['final_sum'].notna()]

In [22]:
data['status'].value_counts()

status
Completed    3327
Name: count, dtype: int64

Осталось 3327 строчек. Количество соотвествует числу транзакций 

Переводим данные в нормальный вид. Считаем день, неделя, месяц, год транзакции

In [23]:
data['action_date'] = pd.to_datetime(data['action_date'], errors='coerce')

In [24]:
data['action_date_month'] = data['action_date'].dt.to_period('M')

In [25]:
data['action_date_year'] = data['action_date'].dt.year

In [26]:
data['action_date_week'] = data['action_date'].dt.to_period('W')

In [27]:
# Курсы валют на 11 февраля 2025 года
exchange_rates = {
    'USD': 96.7821,   # 1 доллар США = 96.7821 рубля
    'EUR': 100.4991,  # 1 евро = 100.4991 рубля
    'BYN': 28.6227    # 1 белорусский рубль = 28.6227 рубля
}

# Функция для конвертации суммы в рубли
def convert_to_rub(row):
    currency = row['operation_currency']
    amount = row['final_sum']
    if currency == 'RUB':
        return amount
    elif currency in exchange_rates:
        return amount * exchange_rates[currency]
    else:
        # Если валюта не распознана, можно вернуть NaN или оставить сумму без изменений
        return amount

# Применяем функцию к DataFrame
data['amount_in_rub'] = data.apply(convert_to_rub, axis=1)

# Обновляем колонку 'operation_currency' на 'RUB'
data['operation_currency'] = 'RUB'


Данные по валюте переводим в рубли 

4. Анализ данных

In [28]:
data['action_date_month'].nunique()

7

В данных представлено семь месяцев 

Далее готовим данные для построения дашборда

Количество пользователей

In [29]:
user_count = data['customer'].nunique()

Результирующая оплата

In [30]:
final_revenue = data['final_sum'].sum().astype(int)

Число транзакций 

In [31]:
final_transaction = data['operation_sum'].sum()

Финальная комиссия 

In [32]:
final_comission = data['comission_sum'].sum()

Средняя величина транзакции 

In [33]:
mean_transaction = data['operation_sum'].mean()

Медиана транзакции 

In [34]:
median_transaction = data['operation_sum'].median()

Количество заплативших пользователей

In [35]:
user_pay_count = data.query('type == "Оплата"')['customer'].nunique()

Считаем первый месяц активности для всех пользователей

In [36]:
data['first_month'] = data['customer'].map(
    data.groupby('customer')['action_date'].min().dt.to_period('M')
)

Кумулитивный рост пользователей по месяцам

In [37]:
users_count_per_month = (
    data
    .groupby('first_month')['customer']
    .nunique()
    .cumsum()
    .reset_index()
)


Кумулитивный рост выручки по месяцам

In [38]:
money_per_month = (
    data.groupby('first_month')['final_sum']
    .sum()
    .round()
    .astype(int)
    .cumsum()
    .reset_index()
)

Рост средней выручки по месяцам

In [39]:
mean_revenue_per_month = data.groupby('action_date_month').agg({
    'final_sum': ['mean']
}).reset_index().round(2)


Скользаящее среднее для двум месяцам

In [40]:
# 1. Сгруппируем по месяцу и возьмем среднюю final_sum
monthly_mean = (
    data
    .groupby('action_date_month', as_index=False)['final_sum']
    .mean()
)

# 2. Отсортируем по датам на случай, если месяц идет не по порядку
monthly_mean = monthly_mean.sort_values('action_date_month')

# 3. Вычислим скользящее среднее по окну в 2 месяца
monthly_mean['rolling_mean_3m'] = monthly_mean['final_sum'].rolling(window=2).mean()


In [41]:
monthly_mean

,action_date_month,final_sum,rolling_mean_3m
0,2024-01,485.908711,NaN
1,2024-02,496.327944,491.118327
2,2024-03,960.804917,728.566431
3,2024-04,728.923287,844.864102
4,2024-05,761.466212,745.194750
5,2024-06,817.223014,789.344613
6,2024-07,691.807779,754.515397


5. Когортный анализ 

Определим первую дату для всех клиентов

In [42]:
user_first_tx = (
    data
    .groupby('customer', as_index=False)
    .agg(first_txn_date=('action_date', 'min'))
)

Округлим до месяцов 

In [43]:
user_first_tx['cohort_month'] = user_first_tx['first_txn_date'].dt.month
user_first_tx['cohort_year'] = user_first_tx['first_txn_date'].dt.year

Смерджим с основной таблицей, установив первую дату активности, месяц и год для всех клиентов 

In [44]:
df_merged = data.merge(user_first_tx, on='customer', how='left', validate='many_to_one')

In [45]:
df_merged['txn_year'] = df_merged['action_date'].dt.year
df_merged['txn_month'] = df_merged['action_date'].dt.month

Удалим строчки, где информации о первых датах нет

In [46]:
df_merged = df_merged.dropna(subset=['first_txn_date'])

Определеям лайфтайм по месяцам

In [47]:
df_merged['month_of_life'] =  df_merged['txn_month'].astype(int) - df_merged['cohort_month'].astype(int)

Группируем все данные по месяцу когорты и лайфтайму для дальнейшей обработки 

In [48]:
grouped = df_merged.groupby(['cohort_month', 'month_of_life'])

Считаем метрики 

In [49]:
cohort_metrics = grouped[['month_of_life','id','customer','final_sum']].agg(
    transactions_count=('id','count'),
    active_users=('customer','nunique'),
    total_sum=('final_sum','sum'),
    avg_check=('final_sum', 'mean')
).reset_index()

# Удаляем столбец cohort_month, если не нужен


Считаем метрики для LTV

In [50]:
cohort_metrics['cumulative_sum'] = cohort_metrics.sort_values('month_of_life') \
    .groupby('cohort_month')['total_sum'].cumsum()

Далее идут сводные таблицы для анализа данных

In [51]:
cohort_pivot_active_users = cohort_metrics.pivot(
    index='month_of_life',       # строка – это месяц жизни когорты
    columns='cohort_month',         # столбец – это сама когорта (год-месяц начала)
    values='active_users'      # какое значение хотим видеть
).fillna(0)

In [52]:
cohort_pivot_ltv = cohort_metrics.pivot(
    index='month_of_life',       # строка – это месяц жизни когорты
    columns='cohort_month',         # столбец – это сама когорта (год-месяц начала)
    values='cumulative_sum'      # какое значение хотим видеть
).fillna(0)

In [53]:
cohort_pivot_transactions = cohort_metrics.pivot(
    index='month_of_life',       # строка – это месяц жизни когорты
    columns='cohort_month',         # столбец – это сама когорта (год-месяц начала)
    values='transactions_count'      # какое значение хотим видеть
).fillna(0)

In [54]:
cohort_pivot_total_sum = cohort_metrics.pivot(
    index='month_of_life',       # строка – это месяц жизни когорты
    columns='cohort_month',         # столбец – это сама когорта (год-месяц начала)
    values='total_sum'      # какое значение хотим видеть
).fillna(0)

In [55]:
cohort_pivot_avg_check = round(cohort_metrics).pivot(
    index='month_of_life',       # строка – это месяц жизни когорты
    columns='cohort_month',         # столбец – это сама когорта (год-месяц начала)
    values='avg_check'      # какое значение хотим видеть
).fillna(0)

Готовим данные для проведения анализа retantion_rate и churn_rate

In [56]:
cohort_sizes = (
    user_first_tx
    .groupby('cohort_month', as_index=False)
    .agg(cohort_size=('customer', 'nunique'))  # или count, если customer уже уникален
)


In [57]:
cohort_metrics = cohort_metrics.merge(
    cohort_sizes,
    on='cohort_month', 
    how='left'
)


In [58]:
cohort_metrics['retention_rate'] = (
    cohort_metrics['active_users'] / cohort_metrics['cohort_size']
)
cohort_metrics['retention_rate'] = round(cohort_metrics['retention_rate'], 2)

In [59]:
cohort_metrics['churn_rate'] = (
    1 - cohort_metrics['retention_rate']
)

cohort_metrics['churn_rate'] = round(cohort_metrics['churn_rate'], 2)

In [60]:
pivot_retention = cohort_metrics.pivot(
    index='month_of_life',
    columns='cohort_month',
    values='retention_rate'
).fillna(0)


In [61]:
pivot_churn = cohort_metrics.pivot(
    index='month_of_life',
    columns='cohort_month',
    values='churn_rate'
).fillna(0)


4. Проведение RFM анализа

In [62]:
def prepare_rfm_data(data, analysis_date=None):
    """
    Подготовка данных для RFM-анализа
    
    Parameters:
    -----------
    df : pandas.DataFrame
        Датафрейм с колонками customer_id, transaction_date, amount
    analysis_date : datetime, optional
        Дата, относительно которой проводится анализ
        
    Returns:
    --------
    pandas.DataFrame
        Датафрейм с RFM-метриками для каждого клиента
    """
    if analysis_date is None:
        analysis_date = data['action_date'].max()
    
    # Группировка по клиентам и расчет RFM-метрик
    rfm = data.groupby('customer').agg({
        'action_date': lambda x: (analysis_date - x.max()).days,  # Recency
        'amount_in_rub': ['count', 'sum']  # Frequency & Monetary
    }).reset_index()
    
    # Переименование колонок
    rfm.columns = ['customer', 'recency', 'frequency', 'monetary']
    
    # Обработка выбросов
    for column in ['recency', 'frequency', 'monetary']:
        q1 = rfm[column].quantile(0.25)
        q3 = rfm[column].quantile(0.75)
        iqr = q3 - q1
        upper_bound = q3 + 1.5 * iqr
        rfm[column] = np.where(rfm[column] > upper_bound, upper_bound, rfm[column])
    
    return rfm

# Применяем функцию к нашим данным
rfm_data = prepare_rfm_data(data)

In [63]:
import pandas as pd

def quantile_segmentation(rfm_data, n_segments=3):
    """
    Квантильная сегментация клиентов

    Parameters:
    -----------
    rfm_data : pandas.DataFrame
        Датафрейм с RFM-метриками
    n_segments : int
        Количество сегментов для каждой метрики

    Returns:
    --------
    pandas.DataFrame
        Датафрейм с добавленными сегментами
    """
    rfm = rfm_data.copy()

    # Функция для безопасного разбиения с учетом количества уникальных значений
    def safe_qcut(series, q):
        unique_values = series.nunique()
        if unique_values < q:
            q = unique_values  # Корректируем число квантилей
        bins = pd.qcut(series, q=q, duplicates='drop', retbins=True)[1]  # Получаем границы
        labels = range(len(bins) - 1, 0, -1)  # Генерируем метки динамически
        return pd.qcut(series, q=q, labels=labels, duplicates='drop')

    # Применяем квантильную сегментацию с учетом особенностей данных
    rfm['R'] = safe_qcut(rfm['recency'], n_segments)
    rfm['F'] = safe_qcut(rfm['frequency'], n_segments)
    rfm['M'] = safe_qcut(rfm['monetary'], n_segments)

    # Создаем RFM Score
    rfm['RFM_Score'] = rfm['R'].astype(str) + rfm['F'].astype(str) + rfm['M'].astype(str)

    return rfm

# Применяем сегментацию
rfm_segmented = quantile_segmentation(rfm_data)
rfm_segmented = rfm_segmented[~rfm_segmented['RFM_Score'].str.contains('nan', na=False)]


In [64]:
def plot_rfm_distributions(rfm_data):
    """
    Визуализация распределения RFM-метрик
    """
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    fig.suptitle('Распределение RFM-метрик', fontsize=14)
    
    # Recency
    sns.histplot(data=rfm_data, x='recency', bins=30, ax=axes[0])
    axes[0].set_title('Распределение Recency')
    axes[0].set_xlabel('Дни с последней покупки')
    
    # Frequency
    sns.histplot(data=rfm_data, x='frequency', bins=30, ax=axes[1])
    axes[1].set_title('Распределение Frequency')
    axes[1].set_xlabel('Количество покупок')
    
    # Monetary
    sns.histplot(data=rfm_data, x='monetary', bins=30, ax=axes[2])
    axes[2].set_title('Распределение Monetary')
    axes[2].set_xlabel('Общая сумма покупок')
    
    plt.tight_layout()
    return fig

# Создаем визуализацию
rfm_dist_plot = plot_rfm_distributions(rfm_data)

5. Вычисление DAU, WAU, sticky 


In [65]:
dau_total = (
    data.groupby('action_date').agg({'customer': 'nunique'}).mean()
)

mau_total = (
    data.groupby(['action_date_month', 'action_date_year'])
    .agg({'customer': 'nunique'})
    .mean()
)


wau_total = (
    data.groupby(['action_date_week'])
    .agg({'customer': 'nunique'})
    .mean()
)



In [66]:
dau_total = int(dau_total)

C:\Users\user\AppData\Local\Temp\ipykernel_8044\2701938052.py:1: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  dau_total = int(dau_total)


In [67]:
wau_total = int(wau_total)

C:\Users\user\AppData\Local\Temp\ipykernel_8044\118994206.py:1: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  wau_total = int(wau_total)


In [68]:
mau_total = int(mau_total)

C:\Users\user\AppData\Local\Temp\ipykernel_8044\3225192726.py:1: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  mau_total = int(mau_total)


In [69]:
wau_total = int(wau_total)

In [70]:
sticky  = (dau_total / wau_total * 100)

Ссылка на дашборд

https://rozhkov1922-work-expe-simple-thingssimple-things-rozhkov-zjgdq9.streamlit.app/ 